# はじめに

Databricksでファイルを取り扱う際には、Databricks File System(DBFS)を理解する必要があります。

本ノートブックでは、DBFSの概要をご説明するとともに、具体的な使用例をデモします。

<table>
  <tr><th>作者</th><th>Databricks Japan</th></tr>
  <tr><td>日付</td><td>2021/02/03</td></tr>
  <tr><td>バージョン</td><td>1.0</td></tr>
  <tr><td>クラスター</td><td>7.4ML</td></tr>
</table>
<img style="margin-top:25px;" src="https://sajpstorage.blob.core.windows.net/workshop20210205/databricks-logo-small-new.png" width="140">

# Databricks File System(DBFS)

Databricks File System(DBFS)はDatabricksのワークスペースにマウントされる分散ファイルシステムです。Databricksクラスターから利用することができます。DBFSはクラウドのオブジェクトストレージを抽象化するものであり、以下のメリットをもたらします：<br><br>

- オブジェクトストレージ(S3/Azure Blob Storageなど)追加の認証情報なしにオブジェクトストレージにアクセスすることができる
- ストレージURLではなく、ディレクトリ、ファイルの文法に従ってファイルにアクセスできる
- ファイルはオブジェクトストレージで永続化されるので、クラスタが削除されてもデータが残る

詳細はこちらを参照ください。

[Databricks File System (DBFS) — Databricks Documentation](https://docs.databricks.com/data/databricks-file-system.html)

## DBFS root

DBFSにおけるデフォルトの場所は「DBFS root」と呼びます。DBFS rootにはいくつかのデータが格納されています。<br><br>

- /FileStore: インポートされたデータファイル、生成されたグラフプロット、アップロードされたライブラリが格納されます。[詳細はこちら](https://docs.databricks.com/data/filestore.html)
- /databricks-datasets: サンプルのデータセットが格納されます。[詳細はこちら](https://docs.databricks.com/data/databricks-datasets.html)
- /databricks-results: クエリ結果の全データをダウンロードする際に生成されるファイルが格納されます。
- /databricks/init: クラスタノードのinit scriptが格納されます。
- /user/hive/warehouse: Databricksで管理するHiveテーブルのメタデータ及びテーブルデータが格納されます。

**注意**
マウントポイント`/mnt`に書き込まれるデータはDBFS rootの外となります。DBFS rootに書き込み権限があったとしても、マウントしたオブジェクトストレージにデータを書き込むことをお勧めします。

## UIからDBFSにアクセス

1. 画面左のDataアイコン![](https://sajpstorage.blob.core.windows.net/workshop20210205/data-icon.png)をクリックします。
2. 画面上部の「DBFS」ボタンをクリックすることでDBFSの階層構造を参照できます。 <br>
**注意** 管理者の方によって「DBFS browser」が有効になっていることを確認ください。
![](https://sajpstorage.blob.core.windows.net/workshop20210205/browse.png)

この他にも、CLI、DBFS API(REST)、Databricksファイルシステムユーティリティ、Spark API、ローカルファイルAPIからもDBFSにアクセスできます。

## DBFSとローカルドライバーノードのパス

Databricksでファイルを操作する際には、DBFSにアクセスしているのか、ローカルのクラスタードライバーノードのファイルシステムにアクセスしているのかを意識する必要があります。

ノートブックからファイルシステムにアクセスする際には、`%fs`、`%sh`といったマジックコマンド、Databrikcsファイルシステムユーティリティ`dbutils.fs`などを使用します。

APIやコマンドによって、パスを指定した際、DBFSを参照するのか、ローカルファイルシステムを参照するのかのデフォルトの挙動が異なりますので注意ください。

<table>
  <tr><th>コマンド</th><th>デフォルト</th><th>DBFSへのアクセス</th><th>ローカルファイルシステムへのアクセス</th></tr>
  <tr><td>%fs</td><td>DBFS root</td><td></td><td>パスの先頭に`file:/`を追加</td></tr>
  <tr><td>%sh</td><td>ローカルドライバノード</td><td>パスの先頭に`/dbfs`を追加</td><td></td></tr>
  <tr><td>dbutils.fs</td><td>DBFS root</td><td></td><td>パスの先頭に`file:/`を追加</td></tr>
  <tr><td>pythonのos.コマンド</td><td>ローカルドライバノード</td><td>パスの先頭に`/dbfs`を追加</td><td></td></tr>
  
</table>

In [0]:
%fs ls /tmp

path,name,size
dbfs:/tmp/KnowledgeRepo/,KnowledgeRepo/,0
dbfs:/tmp/Lakehouse/,Lakehouse/,0
dbfs:/tmp/Network.html,Network.html,3837
dbfs:/tmp/POS_CASH_balance.csv,POS_CASH_balance.csv,392703158
dbfs:/tmp/akermany/,akermany/,0
dbfs:/tmp/amrali/,amrali/,0
dbfs:/tmp/antoine.amend@databricks.com/,antoine.amend@databricks.com/,0
dbfs:/tmp/anton.gross@databricks.com/,anton.gross@databricks.com/,0
dbfs:/tmp/application_train.csv,application_train.csv,166133370
dbfs:/tmp/ashley.trainor/,ashley.trainor/,0


In [0]:
%sh ls /dbfs/tmp

KnowledgeRepo
Lakehouse
Network.html
POS_CASH_balance.csv
akermany
amrali
antoine.amend@databricks.com
anton.gross@databricks.com
application_train.csv
ashley.trainor
ashley.trainor@databricks.com
automl
aweaver
c2f8713c11c2_cdx
cam
carrier_claims
carrier_claims.csv
cdl_demo
cf_matrix.png
charles.bernard
checkpoints
cluster-logs
column_data_amundsen.csv
complaints.csv
complaints_invalid
concavity
credit_card_balance.csv
dbperf_zorder_keijiyoshidadatabrickscom
dbperfdemo_keijiyoshidadatabrickscom
debu.sinha
debu.sinha@databricks.com
delta
demo_project
demographics2008.csv
demographics2009.csv
demographics2010.csv
dennyglee
dennylee
diamonds
ecommerce
ehrdemo_keijiyoshidadatabrickscom
flights_delta
flights_parquet
flowers
football
gdelt
geospatial
gkg
hive
hyperleaup
immuta
immuta-config
inpatient.csv
ionut
iot
joe.widen@databricks.com
json_tests
kbarlow
keiji.yoshida
keijiyoshida
keijiyoshidadatabrickscom
lakehouse_demo
lblackwell
lightgbmNativeTemp.model
loan-risk-analysis
loan_by_state.parquet
mlflow
movies
mymodel
patricia.ferreiro@databricks.com
patricia_ferreiro_databricks_com
peter.stern@databricks.com
pipeline-examples
prescriptions.csv
punkt
rafael.ricotta@databricks.com
readings_agg_tmp3
rwe-ehr
ryangordon
sais_eu_19_demo
sar_spacy
seif
silvio
sim_fin_fraud_detection.csv
sparkr_tmp
table_data_amundsen.csv
takaakiyayoidatabrickscom
taxi-csv
test.json
tmp3_7850s5
tomi
tpcds
ufos
umap.html
wordnet
zinc15_250K_2D.csv

In [0]:
%fs ls file:/tmp

path,name,size
file:/tmp/rstudio-rsession/,rstudio-rsession/,4096
file:/tmp/hsperfdata_root/,hsperfdata_root/,4096
file:/tmp/driver-daemon-params,driver-daemon-params,19
file:/tmp/systemd-private-a439691a9ff84cf08a99b1cbeafa06fc-ntp.service-qj52Mh/,systemd-private-a439691a9ff84cf08a99b1cbeafa06fc-ntp.service-qj52Mh/,4096
file:/tmp/.Test-unix/,.Test-unix/,4096
file:/tmp/systemd-private-a439691a9ff84cf08a99b1cbeafa06fc-systemd-resolved.service-4LaPDJ/,systemd-private-a439691a9ff84cf08a99b1cbeafa06fc-systemd-resolved.service-4LaPDJ/,4096
file:/tmp/.XIM-unix/,.XIM-unix/,4096
file:/tmp/custom-spark.conf,custom-spark.conf,328
file:/tmp/chauffeur-daemon-params,chauffeur-daemon-params,22
file:/tmp/chauffeur-daemon.pid,chauffeur-daemon.pid,4


In [0]:
%sh ls /tmp

Rserv
Rtmpjw9zkX
chauffeur-daemon-params
chauffeur-daemon.pid
chauffeur-env.sh
custom-spark.conf
d3.v3.min.js
driver-daemon-params
driver-daemon.pid
driver-env.sh
hsperfdata_root
rstudio-rsession
systemd-private-a439691a9ff84cf08a99b1cbeafa06fc-apache2.service-qkFelB
systemd-private-a439691a9ff84cf08a99b1cbeafa06fc-ntp.service-qj52Mh
systemd-private-a439691a9ff84cf08a99b1cbeafa06fc-systemd-resolved.service-4LaPDJ
tmp.uXxOyEjO0i

# デモ

FileStoreはファイルを保存したファイルをブラウザから直接参照できる特別なフォルダです。以下のような使い方が可能です。<br><br>

1. HTMLやJavaScriptを保存してブラウザから直接アクセスする。アクセスする際には`DisplayHTML`を使う。
2. 出力結果を保存してローカルのデスクトップにファイルを保存する。

## `displayHTML()`で使うJavaScriptライブラリをダウンロードし、一旦ドライバのローカルディスクに保存します。

In [0]:
%scala
import sys.process._

import sys.process._

In [0]:
%scala
"sudo apt-get -y install wget" !!

warning: there was one feature warning; for details, enable `:setting -feature' or `:replay -feature'
res2: String =
"Reading package lists...
Building dependency tree...
Reading state information...
wget is already the newest version (1.19.4-1ubuntu2.2).
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.
"

In [0]:
%scala
"wget -P /tmp http://d3js.org/d3.v3.min.js" !!

--2021-03-04 08:45:07-- http://d3js.org/d3.v3.min.js
Resolving d3js.org (d3js.org)... 172.67.73.126, 104.26.6.30, 104.26.7.30, ...
Connecting to d3js.org (d3js.org)|172.67.73.126|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://d3js.org/d3.v3.min.js [following]
--2021-03-04 08:45:07-- https://d3js.org/d3.v3.min.js
Connecting to d3js.org (d3js.org)|172.67.73.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/javascript]
Saving to: ‘/tmp/d3.v3.min.js.1’

 0K .......... .......... .......... .......... .......... 4.81M
 50K .......... .......... .......... .......... .......... 4.65M
 100K .......... .......... .......... .......... ........ 18.4M=0.02s

2021-03-04 08:45:08 (6.23 MB/s) - ‘/tmp/d3.v3.min.js.1’ saved [151725]

warning: there was one feature warning; for details, enable `:setting -feature' or `:replay -feature'
res3: String = ""

## `file:/tmp`のファイル一覧を表示し、ローカルディスクにファイルが保存されたことを確認します。

In [0]:
%scala
display(dbutils.fs.ls("file:/tmp/d3.v3.min.js"))

path,name,size
file:/tmp/d3.v3.min.js,d3.v3.min.js,151725


## ローカルの`/tmp`に保存されているファイルをブラウザから直接参照できるように、`/FileStore/customjs`にコピーします

In [0]:
%scala
dbutils.fs.mkdirs("/FileStore/customjs")
dbutils.fs.cp("file:/tmp/d3.v3.min.js", "/FileStore/customjs/d3.v3.min.js")

res5: Boolean = true

## `/FileStore/customjs`のファイル一覧を表示し、ファイルがコピーされたことを確認します

In [0]:
%scala
display(dbutils.fs.ls("/FileStore/customjs"))

path,name,size
dbfs:/FileStore/customjs/d3.v3.min.js,d3.v3.min.js,151725


## 保存したJavaScriptライブラリをブラウザから参照する際には、`/files`から参照することになるので、パスは`/files/customjs/d3.v3.min.js`となります

In [0]:
%scala
displayHTML(s"""
<!DOCTYPE html>
<meta charset="utf-8">
<body>
<script src="/files/customjs/d3.v3.min.js"></script>
<script>
var width = 200
var height = 200
var vertices = d3.range(100).map(function(d) {
  return [Math.random() * width, Math.random() * height];
});

var lineFunction = d3.svg.line()
                         .x(function(d) { return d[0]; })
                         .y(function(d) { return d[1]; })
                         .interpolate("linear");

//The SVG Container
var svgContainer = d3.select("body").append("svg")
                                    .attr("width", 200)
                                    .attr("height", 200);

//The line SVG Path we draw
var lineGraph = svgContainer.append("path")
                            .attr("d", lineFunction(vertices))
                            .attr("stroke", "blue")
                            .attr("stroke-width", 2)
                            .attr("fill", "none");
</script>
  """)

<!DOCTYPE html>